# Load Packages

In [ ]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_prediction_df_dict, save_pi_df_dict
from src.file_manager.save_load_scaler import load_scaler
from src.pi_methods.knn import knn_prediction_interval
from src.pi_methods.weighted import weighted_prediction_interval
from src.pi_methods.cond_gaussian import cond_gauss_prediction_interval
from src.pi_methods.gaussian_copula import gauss_copula_prediction_interval
from src.pi_methods.conformal_prediction import conformal_prediction_interval
from src.misc import create_folder
from ue_pi_dicts import ue_dict
from seed_file import seed

# seed = 2023
data_label = "mimic"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_processed_folder = join(fp_project_folder, "processed_data", data_label)
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")

# Seed filepaths
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)
fp_cur_pi_predictions_folder = join(fp_pi_predictions, str(seed))
create_folder(fp_cur_pi_predictions_folder)

pred_file_names =  ["rue", "gpr", "infernoise", "der", "bnn"]

# Load Data

In [ ]:
split_dict = joblib.load(join(fp_processed_folder, "mimic_split_dict.joblib"))
split_dict["target_cols"] = {
    time_label: time_info["outputs"] for time_label, time_info in split_dict.items()
}
split_dict["feat_cols"] = ['ABPdias (mmHg)_mean Min0', 'RESP (bpm)_mean Min0', 'ABPmean (mmHg)_mean Min0', 'ABPsys (mmHg)_mean Min0', 'SpO2 (%)_mean Min0', 'HR (bpm)_mean Min0', 'ABPdias (mmHg)_std Min0', 'RESP (bpm)_std Min0', 'ABPmean (mmHg)_std Min0', 'ABPsys (mmHg)_std Min0', 'SpO2 (%)_std Min0', 'HR (bpm)_std Min0', 'ABPdias (mmHg)_mean Min1', 'RESP (bpm)_mean Min1', 'ABPmean (mmHg)_mean Min1', 'ABPsys (mmHg)_mean Min1', 'SpO2 (%)_mean Min1', 'HR (bpm)_mean Min1', 'ABPdias (mmHg)_std Min1', 'RESP (bpm)_std Min1', 'ABPmean (mmHg)_std Min1', 'ABPsys (mmHg)_std Min1', 'SpO2 (%)_std Min1', 'HR (bpm)_std Min1', 'ABPdias (mmHg)_mean Min2', 'RESP (bpm)_mean Min2', 'ABPmean (mmHg)_mean Min2', 'ABPsys (mmHg)_mean Min2', 'SpO2 (%)_mean Min2', 'HR (bpm)_mean Min2', 'ABPdias (mmHg)_std Min2', 'RESP (bpm)_std Min2', 'ABPmean (mmHg)_std Min2', 'ABPsys (mmHg)_std Min2', 'SpO2 (%)_std Min2', 'HR (bpm)_std Min2', 'ABPdias (mmHg)_mean Min3', 'RESP (bpm)_mean Min3', 'ABPmean (mmHg)_mean Min3', 'ABPsys (mmHg)_mean Min3', 'SpO2 (%)_mean Min3', 'HR (bpm)_mean Min3', 'ABPdias (mmHg)_std Min3', 'RESP (bpm)_std Min3', 'ABPmean (mmHg)_std Min3', 'ABPsys (mmHg)_std Min3', 'SpO2 (%)_std Min3', 'HR (bpm)_std Min3', 'ABPdias (mmHg)_mean Min4', 'RESP (bpm)_mean Min4', 'ABPmean (mmHg)_mean Min4', 'ABPsys (mmHg)_mean Min4', 'SpO2 (%)_mean Min4', 'HR (bpm)_mean Min4', 'ABPdias (mmHg)_std Min4', 'RESP (bpm)_std Min4', 'ABPmean (mmHg)_std Min4', 'ABPsys (mmHg)_std Min4', 'SpO2 (%)_std Min4', 'HR (bpm)_std Min4']

# Load All Predictions

In [ ]:
scaler = load_scaler(join(fp_processed_folder, "scaler.pkl"))
pred_df_dict = load_prediction_df_dict(
    split_dict, fp_cur_predictions_folder, pred_file_names=pred_file_names) 

# Add KNN PI

In [ ]:
# Apply knn_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = knn_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )


# Add Weighted Percentile PI

In [ ]:
# Apply weighted_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = weighted_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add Cond Gauss PI

In [ ]:
# Apply cond_gauss_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = cond_gauss_prediction_interval(
        df_val=val_df, df_test=test_df, predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add Gaussian Copula PI

In [ ]:
# Apply weighted_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = gauss_copula_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

# Add CP

In [ ]:
# Apply conformal prediction to all UEs
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    for ue_label, ue_info in ue_dict.items():
        val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        pred_df_dict[time_label]["test_df"] = conformal_prediction_interval(
            df_val=val_df, df_test=test_df, 
            predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
            scaler=scaler, seed=seed
        )

# Export df dict

In [ ]:
save_pi_df_dict(pred_df_dict, fp_cur_pi_predictions_folder)